In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [77]:
import os 
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

filename = "/content/drive/MyDrive/EE626_Project/cropped_data/"
images = os.path.join(filename,"cropped_dir/")
annotate = os.path.join(filename,"annotations.txt")

annot = np.loadtxt(annotate,dtype="str")
# print(annot[:,0], annot[:,1])

IMG_SIZE = 128

training_data = []

for img,label in zip(annot[:,0],annot[:,1]):
  img_array = cv.imread(filename+img,0)
  img_resize = cv.resize(img_array, (IMG_SIZE, IMG_SIZE))
  img_reshape = np.reshape(img_resize,(128,128,1))
  training_data.append([img_reshape, label])
  # print(img_array.shape)
  # cv2_imshow(img_resize)
  # cv.waitKey(0)
  

np.save(filename+"training_data.txt",training_data)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [14]:
import numpy as np
from google.colab.patches import cv2_imshow

filename = "/content/drive/MyDrive/EE626_Project/cropped_data/"
training_data = np.loadtxt("/content/drive/MyDrive/EE626_Project/cropped_data/training_data.txt")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: loadtxt: Empty input file: "/content/drive/MyDrive/EE626_Project/cropped_data/training_data.txt"
  """


In [78]:
training_data

[[array([[[154],
          [117],
          [119],
          ...,
          [124],
          [159],
          [152]],
  
         [[152],
          [118],
          [118],
          ...,
          [125],
          [161],
          [152]],
  
         [[147],
          [120],
          [119],
          ...,
          [121],
          [164],
          [152]],
  
         ...,
  
         [[119],
          [ 95],
          [114],
          ...,
          [124],
          [146],
          [148]],
  
         [[118],
          [ 96],
          [113],
          ...,
          [125],
          [147],
          [147]],
  
         [[117],
          [ 99],
          [114],
          ...,
          [125],
          [144],
          [149]]], dtype=uint8), 'जल'], [array([[[121],
          [129],
          [130],
          ...,
          [135],
          [137],
          [179]],
  
         [[120],
          [122],
          [132],
          ...,
          [131],
          [144],
          [156]],


In [79]:
X=[]
Y=[]
for x, y in training_data:
  X.append(x)
  Y.append(y)

In [80]:
X_t = np.array(X)
Y_t = np.array(Y)

In [82]:
np.shape(X_t)

(1740, 128, 128, 1)

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_t,Y_t, test_size = 0.2, random_state=0)

In [84]:
import tensorflow as tf
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
Y_train =  tf.convert_to_tensor(Y_train)
Y_test =  tf.convert_to_tensor(Y_test)

In [85]:
print(tf.shape(X_train[0]))

tf.Tensor([128 128   1], shape=(3,), dtype=int32)


In [ ]:
def ctc_loss_function(args):
    """
    CTC loss function takes the values passed from the model returns the CTC loss using Keras Backend ctc_batch_cost function
    """
    y_pred, y_true, input_length, label_length = args 
    # since the first couple outputs of the RNN tend to be garbage we need to discard them, found this from other CRNN approaches
    # I Tried by including these outputs but the results turned out to be very bad and got very low accuracies on prediction 
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(y_true, y_pred, input_length, label_length)

In [ ]:
#https://github.com/qjadud1994/CRNN-Keras
#https://keras.io/examples/image_ocr/

class DataGenerator(keras.callbacks.Callback):
    def __init__(self, img_dirpath, img_w, img_h,
                 batch_size,n,output_labels,max_text_len=15):
        self.img_h = img_h                    #Image Height
        self.img_w = img_w                    #Image Width
        self.batch_size = batch_size          #Batch size of Input
        self.max_text_len = max_text_len      #Maximum Text length of Labels
        
#         self.n =len(self.img_dir)                           #Number of images in img_matrix
        self.n=n
        self.img_dir = img_dirpath[:self.n]     # images list
        self.indexes = list(range(self.n))   #List of indices for each image in img_matrix
        self.cur_index = 0                   #Current index which points to image being loaded 
        self.imgs = np.zeros((self.n, self.img_h, self.img_w))
        self.texts =  output_labels[:self.n]                  #List of Ground Truth Label texts

   
    def build_data(self):
        """
        Build The Image Data
        """
        print(self.n, " Image Loading start...")
        for i, img_file in enumerate(self.img_dir):
            img = cv2.imread(img_file)
            img = img[:,:,1]                               #Extracting Single Channel Image
            img = cv2.resize(img, (self.img_w, self.img_h))
            img = img /255
            self.imgs[i, :, :]= img
            if i%10000==0:
                print("Loaded Images: ",i)
           
        print("Number of Texts matches with Total Number of Images :",len(self.texts) == self.n)
        print(self.n, " Image Loading finish...")


    def next_data(self): 
        """
        Returns image and text data pointed by the current index
        """
        self.cur_index += 1
        #If current index becomes more than the number of images, make current index 0 
        #and shuffle the indices list for random picking of image and text data
        if self.cur_index >= self.n:
            self.cur_index = 0
            random.shuffle(self.indexes)
        return self.imgs[self.indexes[self.cur_index]], self.texts[self.indexes[self.cur_index]]

    def next_batch(self):
        """
        Creates a batch of images images and text data equal to the batch_size,
        computes the parameters needed for CTC and returns the inputs to the Model
        """
        while True:
            X_data = np.ones([self.batch_size, self.img_w, self.img_h, 1])  #Single channel Gray Size Scale images for input
            #Initilizing with -1 to aid for padding labels of different lengths
            Y_data = np.ones([self.batch_size, self.max_text_len])* -1        #Text labels for input
           #input_length for CTC which is the number of time-steps of the RNN output
            input_length = np.ones((self.batch_size, 1)) * 40
            label_length = np.zeros((self.batch_size, 1))                   #label length for CTC
            source_str=[]                                                   #List to store Ground Truth Labels
            for i in range(self.batch_size):
                img, text = self.next_data() #getting the image and text data pointed by current index
                                    #taking transpose of image
                img=img.T
                img = np.expand_dims(img, -1)  #expanding image to have a single channel
                X_data[i] = img
                label=encode_words_labels(text) # encoding label text to integer list and storing in temp label variable
                lbl_len=len(label)
                Y_data[i,0:lbl_len] = label #Storing the label till its length and padding others
                label_length[i] = len(label)
                source_str.append(text) #storing Ground Truth Labels which will be accessed as reference for calculating metrics
            
        #Preparing the input for the Model
            inputs = {
                'img_input': X_data,  
                'ground_truth_labels': Y_data,  
                'input_length': input_length,  
                'label_length': label_length,
                'source_str': source_str  # used for visualization only
            }
            #Preparing output for the Model and intializing to zeros
            outputs = {'ctc': np.zeros([self.batch_size])}  
            yield (inputs, outputs) # Return the Prepared input and output to the Model

In [ ]:
from keras.layers import Input, Conv2D, MaxPool2D, Dense,MaxPooling2D
from keras.layers import AveragePooling2D, Flatten, Activation, Bidirectional
from keras.layers import BatchNormalization, Dropout
from keras.layers import Concatenate, Add, Multiply, Lambda
from keras.layers import UpSampling2D, Reshape
from keras.layers.merge import add,concatenate
from keras.layers import Reshape
from keras.models import Model
from keras.layers.recurrent import LSTM,GRU
import tensorflow as tf

In [ ]:
def Image_text_recogniser_model_1(stage,drop_out_rate=0.35):
    """
    Builds the model by taking in the stage variable which specifes the stage,
    if the stage is training: model takes inputs required for computing ctc_batch_cost function
    else : model takes input as images which is used for prediction
    """
    
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_w, img_h)
    else:
        input_shape = (img_w, img_h, 1)
       
    model_input=Input(shape=input_shape,name='img_input',dtype='float32')

    # Convolution layer 
    model = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(model_input) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max1')(model) 

    model = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(2, 2), name='max2')(model) 

    model = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max3')(model)  

    model = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(model) 
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Conv2D(512, (3, 3), padding='same', name='conv6')(model)
    model=Dropout(drop_out_rate)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = MaxPooling2D(pool_size=(1, 2), name='max4')(model) 

    model = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(model)
    model=Dropout(0.25)(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)    

    # CNN to RNN
    model = Reshape(target_shape=((42, 1024)), name='reshape')(model)  
    model = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(model)  

    # RNN layer
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='sum')(model)
    model=Bidirectional(LSTM(256, return_sequences=True, kernel_initializer='he_normal'), merge_mode='concat')(model)

    # transforms RNN output to character activations:
    model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model) 
    y_pred = Activation('softmax', name='softmax')(model)

    
    labels = Input(name='ground_truth_labels', shape=[max_length], dtype='float32') 
    input_length = Input(name='input_length', shape=[1], dtype='int64') 
    label_length = Input(name='label_length', shape=[1], dtype='int64') 

    #CTC loss function
    loss_out = Lambda(ctc_loss_function, output_shape=(1,),name='ctc')([y_pred, labels, input_length, label_length]) #(None, 1)

    if stage=='train':
        return model_input,y_pred,Model(inputs=[model_input, labels, input_length, label_length], outputs=loss_out)
    else:
        return Model(inputs=[model_input], outputs=y_pred)

In [ ]:
model_input,y_pred,img_text_recog=Image_text_recogniser_model_1('train')

In [ ]:

def decode_batch(test_func, word_batch):
    """
    Takes the Batch of Predictions and decodes the Predictions by Best Path Decoding and Returns the Output
    """
    out = test_func([word_batch])[0] #returns the predicted output matrix of the model
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = words_from_labels(out_best)
        ret.append(outstr)
    return ret

In [ ]:
def accuracies(actual_labels,predicted_labels,is_train):
    """
    Takes a List of Actual Outputs, predicted Outputs and returns their accuracy and letter accuracy across
    all the labels in the list
    """
    accuracy=0
    letter_acc=0
    letter_cnt=0
    count=0
    for i in range(len(actual_labels)):
        predicted_output=predicted_labels[i]
        actual_output=actual_labels[i]
        count+=1
        for j in range(min(len(predicted_output),len(actual_output))):
            if predicted_output[j]==actual_output[j]:
                letter_acc+=1
        letter_cnt+=max(len(predicted_output),len(actual_output))
        if actual_output==predicted_output:
            accuracy+=1
    final_accuracy=np.round((accuracy/len(actual_labels))*100,2)
    final_letter_acc=np.round((letter_acc/letter_cnt)*100,2)
    return final_accuracy,final_letter_acc

In [ ]:
#https://keras.io/examples/image_ocr/
class VizCallback(keras.callbacks.Callback):
    """
    The Custom Callback created for printing the Accuracy and Letter Accuracy Metrics at the End of Each Epoch
    """

    def __init__(self, test_func, text_img_gen,is_train,acc_compute_batches):
        self.test_func = test_func
        self.text_img_gen = text_img_gen
        self.is_train=is_train                #used to indicate whether the callback is called to for Train or Validation Data
        self.acc_batches=acc_compute_batches  # Number of Batches for which the metrics are computed typically equal to steps/epoch

    def show_accuracy_metrics(self,num_batches):
        """
        Calculates the accuracy and letter accuracy for each batch of inputs, 
        and prints the avarage accuracy and letter accuracy across all the batches
        """
        accuracy=0
        letter_accuracy=0
        batches_cnt=num_batches
        while batches_cnt>0:
            word_batch = next(self.text_img_gen)[0]   #Gets the next batch from the Data generator
            decoded_res = decode_batch(self.test_func,word_batch['img_input'])
            actual_res=word_batch['source_str']
            acc,let_acc=accuracies(actual_res,decoded_res,self.is_train)
            accuracy+=acc
            letter_accuracy+=let_acc
            batches_cnt-=1
        accuracy=accuracy/num_batches
        letter_accuracy=letter_accuracy/num_batches
        if self.is_train:
            print("Train Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Train Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")
        else:
            print("Validation Average Accuracy of "+str(num_batches)+" Batches: ",np.round(accuracy,2)," %")
            print("Validation Average Letter Accuracy of "+str(num_batches)+" Batches: ",np.round(letter_accuracy,2)," %")
            
        
    def on_epoch_end(self, epoch, logs={}):
        self.show_accuracy_metrics(self.acc_batches)

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)
model_chk_pt=ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=False,save_weights_only=True,verbose=0, mode='auto', period=2)

In [89]:
import keras
from keras import backend as K
from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Reshape
from keras.layers import Bidirectional, LSTM, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
import tensorflow as tf
import math


window_height = 128  #windown height
window_width = 128   #window width
window_shift = window_width - 2 #window shift

#CNN related configurations
MPoolLayers_ALL = 5	#Nbr of all maxpool layers
MPoolLayers_H = 2	#Nbr of maxpool in horizontal dimension
LastFilters = 512	#Nbr of feature maps at the last conv layer

#LSTM related configurations
NUnits = 256    #Number of units in forward/backward LSTM
NLayers = 3 
#%%
FV = int(window_height / math.pow(2, MPoolLayers_ALL))
NFeatures = FV * LastFilters

#%%
input_data = Input(shape=(window_height, window_width,1))

convolution1 = Conv2D(filters=64, kernel_size=(1,1))(input_data)
convolution1 = BatchNormalization(axis = -1)(convolution1)
convolution1 = Activation("relu")(convolution1)

convolution2 = Conv2D(filters=64, kernel_size=(1,1))(convolution1)
convolution2 = BatchNormalization(axis = -1)(convolution2)
convolution2 = Activation("relu")(convolution2)

pooling1 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(convolution2)

convolution3 = Conv2D(filters=128, kernel_size=(1,1))(pooling1)
convolution3 = BatchNormalization(axis = -1)(convolution3)
convolution3 = Activation("relu")(convolution3)

convolution4 = Conv2D(filters=128, kernel_size=(1,1))(convolution3)
convolution4 = BatchNormalization(axis = -1)(convolution4)
convolution4 = Activation("relu")(convolution4)

pooling2 = MaxPooling2D(pool_size=(2,2), strides=(2,2))(convolution4)

convolution5 = Conv2D(filters=256, kernel_size=(1,1))(pooling2)
convolution5 = BatchNormalization(axis = -1)(convolution5)
convolution5 = Activation("relu")(convolution5)

convolution6 = Conv2D(filters=256, kernel_size=(1,1))(convolution5)
convolution6 = BatchNormalization(axis = -1)(convolution6)
convolution6 = Activation("relu")(convolution6)

convolution7 = Conv2D(filters=256, kernel_size=(1,1))(convolution6)
convolution7 = BatchNormalization(axis = -1)(convolution7)
convolution7 = Activation("relu")(convolution7)

pooling3 = MaxPooling2D(pool_size=(2,1), strides=(2,1))(convolution7)

convolution8 = Conv2D(filters=512, kernel_size=(1,1))(pooling3)
convolution8 = BatchNormalization(axis = -1)(convolution8)
convolution8 = Activation("relu")(convolution8)

convolution9 = Conv2D(filters=512, kernel_size=(1,1))(convolution8)
convolution9 = BatchNormalization(axis = -1)(convolution9)
convolution9 = Activation("relu")(convolution9)

convolution10 = Conv2D(filters=512, kernel_size=(1,1))(convolution9)
convolution10= BatchNormalization(axis = -1)(convolution10)
convolution10 = Activation("relu")(convolution10)

pooling4 = MaxPooling2D(pool_size=(2,1), strides=(2,1))(convolution10)

convolution11 = Conv2D(filters=512, kernel_size=(1,1))(pooling4)
convolution11= BatchNormalization(axis = -1)(convolution11)
convolution11 = Activation("relu")(convolution11)

convolution12 = Conv2D(filters=512, kernel_size=(1,1))(convolution11)
convolution12= BatchNormalization(axis = -1)(convolution12)
convolution12 = Activation("relu")(convolution12)

convolution13 = Conv2D(filters=512, kernel_size=(1,1))(convolution12)
convolution13= BatchNormalization(axis = -1)(convolution13)
convolution13 = Activation("relu")(convolution13)
# print(convolution13.get_shape())
pooling5 = MaxPooling2D(pool_size=(2,1), strides=(2,1))(convolution13)
# print(pooling5.get_shape())
convolution_full = Reshape(target_shape=(LastFilters * FV,32))(pooling5)

bidir_LSTM1 = Bidirectional(LSTM(units = NUnits, return_sequences=True))(convolution_full)
bidir_LSTM2 = Bidirectional(LSTM(units = NUnits, return_sequences=True))(bidir_LSTM1)
model = Bidirectional(LSTM(units = NUnits))(bidir_LSTM2)
model = Dense(num_classes, kernel_initializer='he_normal',name='dense2')(model) 
    y_pred = Activation('softmax', name='softmax')(model)
Model(inputs = input_data, outputs = y_pred).summary()

#%%
# the actual loss calc occurs here despite it not being an internal Keras loss function

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    print(y_pred, labels, input_length, label_length)
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

# #%%
labels = Input(shape=[20], dtype='float32')
input_length = Input(shape=[1], dtype='int64')
label_length = Input(shape=[1], dtype='int64')
# Keras doesn't currently support loss funcs with extra parameters
# so CTC loss is implemented in a lambda layer
loss_out = Lambda(ctc_lambda_func, output_shape=(), name='ctc')([y_pred, labels, input_length, label_length])
loss_out = CTCLayer(name="ctc_loss")(labels, y_pred)
# clipnorm seems to speeds up convergence
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

# cce = tf.keras.losses.CategoricalCrossentropy()
# out  = cce(y_true, y_pred).numpy()

model = Model(inputs=[input_data, labels ,input_length, label_length],output =loss_out )

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
# model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
# model.compile(loss='categorical_crossentropy', optimizer=sgd)



model.fit(X_train,Y_train, batch_size = 1, epochs = 10, validation_data = (X_test,Y_test))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 128, 128, 64)      128       
_________________________________________________________________
batch_normalization_132 (Bat (None, 128, 128, 64)      256       
_________________________________________________________________
activation_25 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 128, 128, 64)      4160      
_________________________________________________________________
batch_normalization_133 (Bat (None, 128, 128, 64)      256       
_________________________________________________________________
activation_26 (Activation)   (None, 128, 128, 64)      0   

ValueError: ignored

In [ ]:
labels = Input(shape=[47], dtype='float32')
input_length = Input(shape=[1], dtype='int64')
label_length = Input(shape=[1], dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])


Tensor("Placeholder:0", shape=(None, 512), dtype=float32) Tensor("Placeholder_1:0", shape=(None, 47), dtype=float32) Tensor("Placeholder_2:0", shape=(None, 1), dtype=int64) Tensor("Placeholder_3:0", shape=(None, 1), dtype=int64)


ValueError: ignored

In [ ]:
y_pred.shape

TensorShape([None, 512])

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional,Dense,Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

n_timesteps=128
model = Sequential()
model.add(Bidirectional(LSTM(150, return_sequences = True),input_length = (len(X_train[0]),1)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train,Y_train, batch_size = 1, epochs = 10, validation_data = (X_test,Y_test))


In [87]:
import os
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Activation, Dropout, Flatten, Dense, BatchNormalization, Input,Conv2D


# model = Sequential()
# model.add(Input(shape=(128, 128,1)))
# model.add(Conv2D(filters=64, kernel_size=(1,1),
# 			activation = "relu"
#       ,input_shape = (128,128,1)
# 			))

# model.add(BatchNormalization())
# #model.add(Dropout(0.25))
# model.add(MaxPooling2D(pool_size=(2, 2),
# 			strides=(2, 2),
# 			padding="same"))
# model.add(Conv2D(filters = 64,
# 			kernel_size = (3,3),
# 			strides = (1,1),
# 			activation = "relu"))
# model.add(BatchNormalization())
# #model.add(MaxPooling2D())
# model.add(MaxPooling2D(pool_size=(2, 2),
# 			strides=(2, 2),
# 			padding="same"))
# #model.add(Dropout(0.25))
# model.add(Conv2D(filters = 64,
# 			kernel_size = (3,3),
# 			strides = 1,
# 			activation = "relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2),
# 			strides=(2, 2),
# 			padding="same"))
# model.add(Conv2D(filters = 32,
# 			kernel_size = (3,3),
# 			strides= 1,
# 			activation = "relu"))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2),
# 			strides=(2, 2),
# 			padding="same"))			
			
#model.add(Dropout(0.2))
'''model.add(Convolution2D(filters = 64,
			kernel_size = (2,2),
			strides= 1,
			activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),
			strides=(2, 2),
			padding="same"))
'''

# model.add(Flatten())

# model.add(Dense(128,
# 		activation = "relu",
# 		kernel_initializer = "uniform"))
# model.add(BatchNormalization())			

# model.add(Dense(64,
# 		activation = "relu",
# 		kernel_initializer = "uniform"))
# model.add(BatchNormalization())			

# model.add(Dense(46,
# 		activation = "softmax",
# 		kernel_initializer = "uniform"))			
			
# model.compile(optimizer = "adam",
# 		loss = "categorical_crossentropy",
# 		metrics = ["accuracy"])
		
# print(model.summary())
model = Sequential()
model.add(Input(shape=(128, 128,1)))

model.add(Conv2D(filters=64, kernel_size=(1,1)))
model.add(BatchNormalization(axis = -1))
model.add(Activation("relu"))

model.add(Conv2D(filters=64, kernel_size=(1,1)))
model.add(BatchNormalization(axis = -1))
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(1,1)))
model.add(BatchNormalization(axis = -1))
model.add(Activation("relu"))

model.add(Conv2D(filters=128, kernel_size=(1,1)))
model.add(BatchNormalization(axis = -1))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())

model.add(Dense(128,
		activation = "relu",
		kernel_initializer = "uniform"))
model.add(BatchNormalization())			

model.add(Dense(64,
		activation = "relu",
		kernel_initializer = "uniform"))
model.add(BatchNormalization())			

model.add(Dense(1,
		activation = "softmax",
		kernel_initializer = "uniform"))			
			
model.compile(optimizer = "adam",
		loss = "categorical_crossentropy",
		metrics = ["accuracy"])
print(model.summary())
res=model.fit(
		X_train,Y_train,
		epochs = 25, batch_size = 32,
		# steps_per_epoch = 2444,
		validation_data = (X_test, Y_test),
		# validation_steps = 432
		)

%matplotlib inline
accu=res.history['accuracy']
val_acc=res.history['val_accuracy']
loss=res.history['loss']
val_loss=res.history['val_loss']

epochs=range(len(accu)) #No. of epochs

import matplotlib.pyplot as plt
plt.plot(epochs,accu,'r',label='Training Accuracy')
plt.plot(epochs,val_acc,'g',label='Testing Accuracy')
plt.legend()
plt.figure()

#Plot training and validation loss per epoch
plt.plot(epochs,loss,'r',label='Training Loss')
plt.plot(epochs,val_loss,'g',label='Testing Loss')
plt.legend()
plt.show()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 128, 128, 64)      128       
_________________________________________________________________
batch_normalization_113 (Bat (None, 128, 128, 64)      256       
_________________________________________________________________
activation_8 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 128, 128, 64)      4160      
_________________________________________________________________
batch_normalization_114 (Bat (None, 128, 128, 64)      256       
_________________________________________________________________
activation_9 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 64, 64, 64)      

UnimplementedError: ignored